In [1]:
! pip install tranformers huggingface_hub
! pip install datasets
! git config --global user.name ''
! git config --global user.email ''
! pip install arabert

ERROR: Could not find a version that satisfies the requirement tranformers (from versions: none)
ERROR: No matching distribution found for tranformers


I am going to use the aragpt-2 large model

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from arabert.preprocess import ArabertPreprocessor

In [17]:
check_point = 'aubmindlab/aragpt2-base'
model = AutoModelForCausalLM.from_pretrained(check_point)
tokenizer = AutoTokenizer.from_pretrained(check_point)
dataset = load_dataset('parquet', data_files="\data\extracted\first_1000_record.parquet") # change the path

dataset

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 49790
    })
})

In [18]:
arabert_pre = ArabertPreprocessor(check_point)

In [19]:
# set the padding token
tokenizer.pad_token = tokenizer.eos_token

In [20]:
def tokenizer_function(examples):
    cleaned_text = [arabert_pre.preprocess(text) for text in examples['text']]

    tokenized_output = tokenizer(
        cleaned_text,
        truncation=True,
        max_length=1000,
        padding='max_length', 
        return_tensors="pt"
    )

    # Convert the tensor outputs back to lists
    tokenized_output = {k: v.tolist() for k, v in tokenized_output.items()}

    # Set labels equal to input_ids
    tokenized_output['labels'] = tokenized_output['input_ids'].copy()

    return tokenized_output


train_data = dataset.map(tokenizer_function, batched=True, remove_columns=['text'])


data_collator = DataCollatorWithPadding(tokenizer)


Map:   0%|          | 0/49790 [00:00<?, ? examples/s]

In [9]:
train_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 248953
    })
})

In [11]:
print(train_data)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 248953
    })
})


In [23]:
training_args = TrainingArguments('test-trainer',
                                 per_device_train_batch_size=16, num_train_epochs=5, save_strategy='epoch',
                                 learning_rate=2e-5, weight_decay=0.01,
                                 push_to_hub=True,
                                 hub_model_id='AraGPT2-IbnTaymiyyah',
                                 hub_token='',  evaluation_strategy='no')


trainer = Trainer(model=model, args = training_args, train_dataset=train_data['train'],
                 data_collator=data_collator, tokenizer=tokenizer)


trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss
500,0.255600
1000,0.259100
1500,0.259300
2000,0.253400
2500,0.250400
3000,0.251200
3500,0.253300
4000,0.251200
4500,0.258200
5000,0.248100


Step,Training Loss
500,0.255600
1000,0.259100
1500,0.259300
2000,0.253400
2500,0.250400
3000,0.251200
3500,0.253300
4000,0.251200
4500,0.258200
5000,0.248100


TrainOutput(global_step=15560, training_loss=0.24515642819490407, metrics={'train_runtime': 15972.738, 'train_samples_per_second': 15.586, 'train_steps_per_second': 0.974, 'total_flos': 1.270481472e+17, 'train_loss': 0.24515642819490407, 'epoch': 5.0})